In [1]:
# load data
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from keras.models import Model
from keras.layers import Flatten, Dense, Input
from keras.layers import Convolution2D, MaxPooling2D, BatchNormalization, Dropout
from keras.layers import GlobalMaxPooling2D, concatenate
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import np_utils
from keras import regularizers


training_path = "../Test_data/train.json"
testing_path = "../Test_data/test.json"

train_data = pd.read_json(training_path)
test_data = pd.read_json(testing_path)


# no third for inc angle
def get_scaled_imgs(df):
    imgs = []
    
    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)
        
        # Rescale
        a = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())
        b = (band_2 - band_2.mean()) / (band_2.max() - band_2.min())
        c = (band_3 - band_3.mean()) / (band_3.max() - band_3.min())

        imgs.append(np.dstack((a, b, c)))    # , c)))

    return np.array(imgs)

X = get_scaled_imgs(train_data)



Y = train_data["is_iceberg"]



X_full = X
Y_full = Y


X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size = 0.25,
                                                    random_state=42)

print("size x_train", len(X_train))
print("size x_test", len(X_test))

print("size y_train", len(y_train))
print("size y_test", len(y_test))

print("size x full", len(X_full))
print("size y full", len(Y_full))


Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


size x_train 1203
size x_test 401
size y_train 1203
size y_test 401
size x full 1604
size y full 1604


In [3]:
# create the model
img_input = Input(shape=(75,75,3))
x = Convolution2D(64, (3,3), activation='relu', padding='same',
                  kernel_regularizer=regularizers.l2(0.1),
                  bias_regularizer=regularizers.l2(0.1), name='block1_conv1')(img_input)
x = Convolution2D(64, (3,3), activation='relu', padding='same',
                  kernel_regularizer=regularizers.l2(0.1),
                  bias_regularizer=regularizers.l2(0.1), name='block1_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)
x = BatchNormalization()(x)

x = Convolution2D(128, (3,3), activation='relu', padding='same',
                  kernel_regularizer=regularizers.l2(0.1),
                  bias_regularizer=regularizers.l2(0.1), name='block2_conv1')(x)
x = Convolution2D(128, (3,3), activation='relu', padding='same',
                  kernel_regularizer=regularizers.l2(0.1),
                  bias_regularizer=regularizers.l2(0.1), name='block2_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)
x = BatchNormalization()(x)

x = Convolution2D(256, (3,3), activation='relu', padding='same',
                  kernel_regularizer=regularizers.l2(0.1),
                  bias_regularizer=regularizers.l2(0.1), name='block3_conv1')(x)
x = Convolution2D(256, (3,3), activation='relu', padding='same',
                  kernel_regularizer=regularizers.l2(0.1),
                  bias_regularizer=regularizers.l2(0.1), name='block3_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)
x = BatchNormalization()(x)

x = Convolution2D(256, (3,3), activation='relu', padding='same',
                  kernel_regularizer=regularizers.l2(0.1),
                  bias_regularizer=regularizers.l2(0.1), name='block4_conv1')(x)
x = Convolution2D(256, (3,3), activation='relu', padding='same',
                  kernel_regularizer=regularizers.l2(0.1),
                  bias_regularizer=regularizers.l2(0.1), name='block4_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)
x = BatchNormalization()(x)

x = Flatten(name='flatten')(x)
x = Dense(512, kernel_regularizer=regularizers.l2(0.1),
          bias_regularizer=regularizers.l2(0.1))(x)
x = Dropout(0.5)(x)
x = Dense(256, kernel_regularizer=regularizers.l2(0.1),
          bias_regularizer=regularizers.l2(0.1))(x)
x = Dropout(0.5)(x)
main_output = Dense(1, activation='sigmoid', name='predictions')(x)

model = Model(inputs=[img_input], outputs= [main_output], name='vgg_inc')
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.00001), metrics=['accuracy'])
model.summary()


def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]
file_path = "model_weights_basic_reg10_full.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=15)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 75, 75, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 75, 75, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 75, 75, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 37, 37, 64)        0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 37, 37, 64)        256       
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 37, 37, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 37, 37, 128)       147584    
__________

In [4]:
# training code
#%% Image data augmentation 
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=20,                      # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,                  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,                 # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,                   # randomly flip images
    vertical_flip=False,                    # randomly flip images
    zoom_range=0.2)                     

datagen.fit(X_train)
model.fit_generator(datagen.flow(X_full, Y_full, shuffle=True,
                    batch_size=32), epochs = 160,
                     verbose= 1, callbacks=callbacks) # validation_data = (X_test, y_test),
                        #callbacks=callbacks)   # validation_data = (X_test, y_test)

Epoch 1/160
51/51 [==============================] - 11s 218ms/step - loss: 247.6931 - acc: 0.5648
Epoch 2/160
 1/51 [..............................] - ETA: 8s - loss: 246.4596 - acc: 0.6562

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/callbacks.py:494: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/callbacks.py:403: RuntimeWarning: Can save best model only with val_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


51/51 [==============================] - 7s 135ms/step - loss: 245.5045 - acc: 0.6060
Epoch 3/160
51/51 [==============================] - 7s 135ms/step - loss: 243.2222 - acc: 0.6279
Epoch 4/160
51/51 [==============================] - 7s 135ms/step - loss: 241.0147 - acc: 0.6434
Epoch 5/160
51/51 [==============================] - 7s 136ms/step - loss: 238.8675 - acc: 0.6564
Epoch 6/160
51/51 [==============================] - 7s 136ms/step - loss: 236.6734 - acc: 0.6776
Epoch 7/160
51/51 [==============================] - 7s 136ms/step - loss: 234.4979 - acc: 0.6845
Epoch 8/160
51/51 [==============================] - 7s 136ms/step - loss: 232.3719 - acc: 0.6990
Epoch 9/160
51/51 [==============================] - 7s 136ms/step - loss: 230.2153 - acc: 0.7053
Epoch 10/160
51/51 [==============================] - 7s 136ms/step - loss: 228.0471 - acc: 0.7199
Epoch 11/160
51/51 [==============================] - 7s 137ms/step - loss: 225.9575 - acc: 0.7125
Epoch 12/160
51/51 [==========

51/51 [==============================] - 7s 138ms/step - loss: 88.6975 - acc: 0.8495
Epoch 86/160
51/51 [==============================] - 7s 137ms/step - loss: 87.2116 - acc: 0.8842
Epoch 87/160
51/51 [==============================] - 7s 137ms/step - loss: 85.7854 - acc: 0.8737
Epoch 88/160
51/51 [==============================] - 7s 137ms/step - loss: 84.3435 - acc: 0.8829
Epoch 89/160
51/51 [==============================] - 7s 137ms/step - loss: 82.9342 - acc: 0.8708
Epoch 90/160
51/51 [==============================] - 7s 137ms/step - loss: 81.5226 - acc: 0.8860
Epoch 91/160
51/51 [==============================] - 7s 138ms/step - loss: 80.1319 - acc: 0.8806
Epoch 92/160
51/51 [==============================] - 7s 137ms/step - loss: 78.7686 - acc: 0.8689
Epoch 93/160
51/51 [==============================] - 7s 139ms/step - loss: 77.4057 - acc: 0.8593
Epoch 94/160
51/51 [==============================] - 7s 137ms/step - loss: 76.0337 - acc: 0.8848
Epoch 95/160
51/51 [=============

In [5]:
# if doing no val set (i.e. final) only
filepath_full = 'model_weights_inc_full_reg10_Fin.hdf5'
model.save(filepath_full)

In [2]:
# from above, need to see the amount of epochs (80 +) that val stops increasing at, then do 
# that in one full go and save the model
filepath_full = 'model_weights_inc_full_reg10_Fin.hdf5'
from keras.models import load_model
import pandas as pd
# if not using val
file_path_use = filepath_full
# if using val
#file_path_use = file_path
inf_model = load_model(file_path_use)

score = inf_model.evaluate(X_full, Y_full, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


X_sub = get_scaled_imgs(test_data)

predicted_test=inf_model.predict(X_sub)

print("len of pred test", len(predicted_test))
print("len of id", len(test_data['id']))

submission = pd.DataFrame()
submission['id']=test_data['id']
submission['is_iceberg']=predicted_test




1604/1604 [==============================] - 3s 2ms/step
Test loss: 19.0173571437
Test accuracy: 0.927057356608
len of pred test 8424
len of id 8424


In [3]:
# do some examination and clipping here first






submission.to_csv('sub_full_basic_reg01_Fin.csv', index=False)